# <center >TAPE

- `transformer` (pretrained available)
- `resnet`
- `lstm`
- `unirep` (pretrained available)
- `onehot` (no pretraining required)
- `trrosetta` (pretrained available)

In [19]:
import numpy as np
import pandas as pd
import altair      as alt

import torch
from tqdm import tqdm_notebook as tqdm
from tape import TAPETokenizer, ProteinBertModel # ProteinLSTMModel

# BERT

In [2]:
# Pfam Family: Hexapep, Clan: CL0536
sequence  = 'GCTVEDRCLIGMGAILLNGCVIGSGSLVAAGALITQ'
tokenizer = TAPETokenizer(vocab='iupac')  # iupac is the vocab for TAPE models, use unirep for the UniRep model
token_ids = torch.tensor([tokenizer.encode(sequence)]).cuda()
token_ids

tensor([[ 2, 11,  7, 23, 25,  9,  8, 21,  7, 15, 13, 11, 16, 11,  5, 13, 15, 15,
         17, 11,  7, 25, 13, 11, 22, 11, 22, 15, 25,  5,  5, 11,  5, 15, 13, 23,
         20,  3]], device='cuda:0')

In [3]:
bert = ProteinBertModel.from_pretrained('bert-base').cuda().eval()

# NOTE: pooled_output is *not* trained for the transformer, do not use
# w/o fine-tuning. A better option for now is to simply take a mean of
# the sequence output
output = bert(token_ids)
sequence_output = output[0]
pooled_output = output[1]

sequence_output.shape, pooled_output.shape

(torch.Size([1, 38, 768]), torch.Size([1, 768]))

# Inferirir Macros

In [4]:
df = pandas.read_csv("../data/macros.csv", index_col="Uniprot Code")
df.head()

,Tipo de Macro,Secuencia
Uniprot Code,,
O28751,AF-1521-like,MEVLFEAKVGDITLKLAQGDITQYPAKAIVNAANKRLEHGGGVAYA...
D3RWS7,AF-1521-like,MEVEVVRELEMDKLKVKLAGGDITKYPAEAIVNAANKYLEHGGGVA...
D2RH24,AF-1521-like,MVVKKFGSVEVVLEKGDITKYPAEAIVNAANKYLEHGGGVALAIAK...
A0A0F7ICE9,AF-1521-like,MKPEVVLRFSGVEVRLVQGDITKYPAEAIVNAANRHLEHGGGVAYA...
A0A075LQ95,AF-1521-like,MNLTELTFGNLTFKLAQGDITKLPAEAIVNAANKYLEHGGGVALAI...


In [5]:
seqs        = df["Secuencia"].values
seqs_tokens =  [torch.tensor([tokenizer.encode(seq)]).cuda() for seq in seqs]

In [6]:
outputs = []

for seq_tokens in tqdm(seqs_tokens):
    with torch.no_grad():
        output = bert(seq_tokens)[1][0].cpu().numpy()
    
    print(output.shape)
    outputs.append(output)

<ipython-input-6-48d4cd07571d>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for seq_tokens in tqdm(seqs_tokens):


(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)

In [11]:
outputs_np = np.array(outputs)
outputs_np.shape

(165, 768)

In [13]:
#np.save("macros_bert.npy", outputs_np)